<a href="https://colab.research.google.com/github/CarlTeapot/Walmart-Recruiting/blob/main/model_experiment_N_Beats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow==2.2.2
!pip install dagshub

In [ ]:
!pip install setuptools==65.5.0


In [3]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/ML PROEQTI/clean_train_data.csv')

In [5]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import mlflow
import mlflow.sklearn



df = train_df.copy()
df['Type'] = df['Type'].astype('category')
df['Season'] = df['Season'].astype('category')
X = df.drop(columns=['Weekly_Sales'])
y = df['Weekly_Sales']

split_index = int(len(X) * 0.8)

X_train = X[:split_index]
X_val = X[split_index:]

y_train = y[:split_index]
y_val = y[split_index:]

In [6]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import lightgbm as lgb

class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.drop(columns=self.columns_to_drop)

preprocessing = Pipeline(steps=[
    ('drop_columns', DropColumns(columns_to_drop=[
        'Date',
    ])),
])

In [ ]:
from sklearn.metrics import mean_absolute_error

def wmae(y_true, y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

# Predictions
y_train_pred = pipeline.predict(X_train)
y_val_pred = pipeline.predict(X_val)

# Weights
train_weights = X_train['IsHoliday'].replace(1, 5).replace(0, 1)
val_weights = X_val['IsHoliday'].replace(1, 5).replace(0, 1)

# WMAE
wmae_train = wmae(y_train, y_train_pred, train_weights)
wmae_val = wmae(y_val, y_val_pred, val_weights)

print(f"Train WMAE: {wmae_train}")
print(f"Validation WMAE: {wmae_val}")

/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:1085: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  with _raise_or_warn_if_not_fitted(self):
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:1085: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  with _raise_or_warn_if_not_fitted(self):


Train WMAE: 3800.3154786841214
Validation WMAE: 4542.555695923023


In [ ]:

name = "XGBoost with train test split and lag features"


with mlflow.start_run(run_name = name):
        mlflow.sklearn.log_model(pipeline, "best_model")
        mlflow.log_metric("Validation WMAE ", wmae_val)
        mlflow.log_metric("Train WMAE", wmae_train)


2025/08/01 22:19:01 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-02-28; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'rapids-dask-dependency'}


In [ ]:
import mlflow
import dagshub

dagshub.init(repo_owner='CarlTeapot', repo_name='Walmart-Recruiting', mlflow=True)

Accessing as CarlTeapot

Initialized MLflow to track repo "CarlTeapot/Walmart-Recruiting"

Repository CarlTeapot/Walmart-Recruiting initialized!

In [ ]:
from darts import TimeSeries
from darts.models import NBEATSModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import mape, mae, rmse
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

def prepare_nbeats_data(train_df):
    df = train_df.copy()

    # Ensure Date is datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Fill NaNs with 0
    df.fillna(0, inplace=True)

    df['Weekly_Sales'] = np.log1p(df['Weekly_Sales'])

    # Sort by date
    df = df.sort_values('Date').reset_index(drop=True)

    # Handle categorical variables
    label_encoders = {}
    categorical_cols = df.select_dtypes(include=['category', 'object']).columns
    categorical_cols = [col for col in categorical_cols if col != 'Date']

    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le

    return df, label_encoders

def wmae(y_true, y_pred, weights):
    """Calculate Weighted Mean Absolute Error"""
    if np.sum(weights) == 0:
        return np.nan
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)

print("Preparing data for N-BEATS...")
df, label_encoders = prepare_nbeats_data(train_df)

split_index = int(len(df) * 0.8)
train_data = df[:split_index].copy()
val_data = df[split_index:].copy()


# Main target series
train_ts = TimeSeries.from_dataframe(
    train_data,
    time_col='Date',
    value_cols='Weekly_Sales',
    freq='W'
)

val_ts = TimeSeries.from_dataframe(
    val_data,
    time_col='Date',
    value_cols='Weekly_Sales',
    freq='W'
)

covariate_cols = [col for col in df.columns if col not in ['Date', 'Weekly_Sales']]
print(f"Available covariates: {covariate_cols}")

if covariate_cols:
    train_covariates = TimeSeries.from_dataframe(
        train_data,
        time_col='Date',
        value_cols=covariate_cols,
        freq='W'
    )

    val_covariates = TimeSeries.from_dataframe(
        val_data,
        time_col='Date',
        value_cols=covariate_cols,
        freq='W'
    )

    full_covariates = TimeSeries.from_dataframe(
        df,
        time_col='Date',
        value_cols=covariate_cols,
        freq='W'
    )
else:
    train_covariates = None
    val_covariates = None
    full_covariates = None

# 7. Scale the data
print("Scaling data...")
scaler = Scaler()
train_ts_scaled = scaler.fit_transform(train_ts)

# 8. Configure and train N-BEATS model
print("Configuring N-BEATS model...")

# Start MLflow run
with mlflow.start_run(run_name="N-BEATS Weekly Sales Forecasting"):

    input_chunk_length = 52          # 1 year of history
    output_chunk_length = 1          # predict 1 step ahead (weekly)
    num_stacks = 4                   # balanced depth
    num_blocks = 2                   # 2 blocks per stack
    num_layers = 2                   # moderate depth inside each block
    layer_widths = 256               # smaller width to reduce overfitting
    n_epochs = 100                   # enough for convergence
    batch_size = 64                  # standard for stability and speed


    mlflow.log_param("model_type", "N-BEATS")
    mlflow.log_param("input_chunk_length", input_chunk_length)
    mlflow.log_param("output_chunk_length", output_chunk_length)
    mlflow.log_param("num_stacks", num_stacks)
    mlflow.log_param("num_blocks", num_blocks)
    mlflow.log_param("num_layers", num_layers)
    mlflow.log_param("layer_widths", layer_widths)
    mlflow.log_param("n_epochs", n_epochs)
    mlflow.log_param("batch_size", batch_size)

    nbeats_model = NBEATSModel(
        input_chunk_length=input_chunk_length,
        output_chunk_length=output_chunk_length,
        num_stacks=num_stacks,
        num_blocks=num_blocks,
        num_layers=num_layers,
        layer_widths=layer_widths,
        n_epochs=n_epochs,
        batch_size=batch_size,
        random_state=42,
        save_checkpoints=True,
        force_reset=True,
        pl_trainer_kwargs={
            "accelerator": "auto",
            "enable_model_summary": False
        }
    )

    print("Training N-BEATS model...")
    print("This may take several minutes...")

    if train_covariates is not None:
        nbeats_model.fit(
            series=train_ts_scaled,
            past_covariates=train_covariates,
            verbose=True
        )
    else:
        nbeats_model.fit(
            series=train_ts_scaled,
            verbose=True
        )

    print("Training completed!")

    print("Making predictions...")

    n_val_periods = len(val_ts)

    if full_covariates is not None:
        # Use covariates for prediction
        predictions_scaled = nbeats_model.predict(
            n=n_val_periods,
            series=train_ts_scaled,
            past_covariates=full_covariates
        )
    else:
        predictions_scaled = nbeats_model.predict(
            n=n_val_periods,
            series=train_ts_scaled
        )

    # Inverse transform predictions
    predictions = scaler.inverse_transform(predictions_scaled)

    # Convert to numpy arrays for evaluation
    y_true = val_ts.values().flatten()
    y_pred = predictions.values().flatten()

    # Ensure same length
    min_len = min(len(y_true), len(y_pred))
    y_true = y_true[:min_len]
    y_pred = y_pred[:min_len]

    # 10. Calculate metrics including WMAE
    print("Calculating metrics...")

    # Standard metrics
    mae_score = mae(val_ts[:min_len], predictions[:min_len])
    rmse_score = rmse(val_ts[:min_len], predictions[:min_len])
    mape_score = mape(val_ts[:min_len], predictions[:min_len])

    try:
        # Get holiday weights for validation period
        val_dates = val_data['Date'].iloc[:min_len]

        # Create weights based on IsHoliday if available
        if 'IsHoliday' in val_data.columns:
            holiday_flags = val_data['IsHoliday'].iloc[:min_len].values
            val_weights = np.where(holiday_flags == 1, 5, 1)  # 5x weight for holidays
            print("Using holiday weights for WMAE calculation.")
        else:
            val_weights = np.ones(min_len)
            print("No holiday information available. Using uniform weights.")

        wmae_score = wmae(y_true, y_pred, val_weights)

    except Exception as e:
        print(f"Error calculating WMAE: {e}. Using MAE instead.")
        wmae_score = np.mean(np.abs(y_true - y_pred))

    # Log metrics
    mlflow.log_metric("MAE", mae_score)
    mlflow.log_metric("RMSE", rmse_score)
    mlflow.log_metric("MAPE", mape_score)
    mlflow.log_metric("WMAE", wmae_score)

    # Print results
    print(f"\nN-BEATS Model Performance:")
    print(f"MAE: {mae_score:.2f}")
    print(f"RMSE: {rmse_score:.2f}")
    print(f"MAPE: {mape_score:.2f}%")
    print(f"WMAE: {wmae_score:.2f}")

    # 11. Create predictions dataframe for analysis
    predictions_df = pd.DataFrame({
        'Date': val_data['Date'].iloc[:min_len],
        'Actual': y_true,
        'Predicted': y_pred,
        'Error': y_true - y_pred,
        'Absolute_Error': np.abs(y_true - y_pred)
    })

    if 'IsHoliday' in val_data.columns:
        predictions_df['IsHoliday'] = val_data['IsHoliday'].iloc[:min_len].values

    print(f"\nPredictions sample:")
    print(predictions_df.head(10))

    # 12. Save model (optional)
    try:
        # Save the trained model
        nbeats_model.save("nbeats_weekly_sales__model.pkl")
        mlflow.log_artifact("nbeats_weekly_sales_model.pkl")
        print("Model saved successfully!")

    except Exception as e:
        print(f"Error saving model: {e}")

    print("\nN-BEATS model training and evaluation complete!")

# 13. Function to make future predictions
def make_future_predictions(model, scaler, train_ts_scaled, n_periods=12, covariates=None):
    """Make future predictions with the trained N-BEATS model"""

    if covariates is not None:
        future_predictions_scaled = model.predict(
            n=n_periods,
            series=train_ts_scaled,
            past_covariates=covariates
        )
    else:
        future_predictions_scaled = model.predict(
            n=n_periods,
            series=train_ts_scaled
        )

    # Inverse transform
    future_predictions = scaler.inverse_transform(future_predictions_scaled)

    return future_predictions

Preparing data for N-BEATS...
Available covariates: ['Store', 'Dept', 'IsHoliday', 'Size', 'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Month', 'Year', 'Week', 'Holiday_in_1_week_lag', 'Weeks_until_next_holiday']
Scaling data...
Configuring N-BEATS model...
Training N-BEATS model...
This may take several minutes...


Training: |          | 0/? [00:00<?, ?it/s]

### N-BEATS Model WMAE

The following cell calculates and prints the Weighted Mean Absolute Error (WMAE) for the N-BEATS model. The WMAE is a custom metric that assigns a higher weight to errors on holiday sales, making it a more relevant metric for this business problem.

In [56]:
# Calculate and print the WMAE for the N-BEATS model
val_weights = val_data['IsHoliday'].apply(lambda x: 5 if x else 1)
wmae_score = wmae(y_true, y_pred, val_weights)

print(f"N-BEATS Model WMAE: {wmae_score:.2f}")

N-BEATS Model WMAE: 49281491.53


In [31]:
from nbeats_pytorch.model import NBeatsNet

In [30]:
!pip install nbeats-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.8
    Uninstalling protobuf-4.25.8:
      Successfully uninstalled protobuf-4.25.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.2.2 requires packaging<24, but you have packaging 25.0 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.0 which is incompatible.
google-cloud-language 2.17.2 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<7.0.0,>=3.20.2, but you have protobuf 3.20.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.0 which is incompatible.
google-cloud-datastore 2.21.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.

In [60]:
!pip install neuralprophet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.7 MB/s eta 0:00:00
